In [ ]:
# Connecting with Spotify API
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Initialize spotipy with Spotify API credentials
client_id = '5093ad0ad45f435ca7ce43b2afd406b9'  # Replace with your client ID
client_secret = 'f151962f414144aa85494be81ccd8e20'  # Replace with your client secret
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
# Fetching data from Spotify Million Playlist dataset
import json

# Load JSON data from file
with open('/data/challenge_set.json', 'r') as file: # Replace with local dataset path
    data = json.load(file)

# Initialize an empty list to collect all track data
all_tracks = []

# Loop through each playlist in the dataset
for playlist in data['playlists']:
    for track in playlist['tracks']:
        # Add playlist-level information to each track record
        track_info = {
            'playlist_name': playlist.get('name', 'Unknown'),
            'playlist_pid': playlist['pid'],
            'playlist_num_tracks': playlist['num_tracks'],
            'track_pos': track['pos'],
            'artist_name': track['artist_name'],
            'track_uri': track['track_uri'],
            'artist_uri': track['artist_uri'],
            'track_name': track['track_name'],
            'album_uri': track['album_uri'],
            'duration_ms': track['duration_ms'],
            'album_name': track['album_name']
        }
        all_tracks.append(track_info)

# Convert the list of track dictionaries to a DataFrame
df_spotify = pd.DataFrame(all_tracks)

# Display the first few rows of the DataFrame to verify
print(df_spotify.head())

In [ ]:
# Creating new dataset for getting lyrics of unique songs
df_lyrics = df_spotify.drop_duplicates(subset=['track_uri']).drop(['playlist_name', 'playlist_pid', 'playlist_num_tracks', 'track_pos',
       'artist_uri', 'album_uri', 'duration_ms', 'album_name'], axis=1).reset_index()#inplace=True)

In [ ]:
# Function to fetch artist name and track name
def fetch_tracks_details(track_uris):
    try:
        tracks_info = sp.tracks(track_uris)
        results = []
        for track in tracks_info['tracks']:
            if track:  # Check if track details are successfully retrieved
                artist_name = track['artists'][0]['name'] if track['artists'] else 'Unknown'
                track_name = track['name'] if track else 'Unknown'
                results.append((artist_name, track_name))
            else:
                results.append(('Unknown', 'Unknown'))
        return results
    except Exception as e:
        print(f"Error processing batch: {str(e)}")
        return [('Unknown', 'Unknown')] * len(track_uris)

In [ ]:
# Apply batch processing
batch_size = 50
artist_names = []
track_names = []

for i in range(0, len(df_lyrics['track_uri']), batch_size):
    batch_uris = df_lyrics['track_uri'][i:i + batch_size].tolist()
    batch_results = fetch_tracks_details(batch_uris)
    artist_names.extend([res[0] for res in batch_results])
    track_names.extend([res[1] for res in batch_results])

df_lyrics['alb_name'] = artist_names
df_lyrics['trk_name'] = track_names

In [ ]:
# Save the updated DataFrame
df_lyrics.to_csv('df_lyrics_updated.csv', index=False)